# Resilient Object Streaming in AIStore

The following demo shows how to use `ObjectFile` (`aistore.sdk.obj.obj_file.object_file`) to stream large objects amidst potential instances of `ChunkedEncodingError` due to momentary issues with the cluster or its availability mid-read:

In [ ]:
# Step 0: Import Necessary Libraries

import os
import requests
import shutil
import tarfile
import urllib3

from aistore.sdk.client import Client

In [ ]:
# Step 1: Initialize AIStore Client with Retries

AIS_ENDPOINT = "http://localhost:8080"

# Define custom retry logic for requests to AIS. This will also be used when re-establishing streams (in the case of object.get().as_file()).
# If you want to retry in the case of total pod failure, be sure to force retries on specific HTTP response codes that are not typically retried
# In particular, 400 and 404 are what you might see as the client attempts to redirect requests to an object on a missing target
# The timing on each retry is determined by (backoff_factor * 2^retry_count) -- here the last and longest retry waits 512 seconds
retry = urllib3.Retry(total=10, backoff_factor=0.5, status_forcelist=[400, 404])
client = Client(AIS_ENDPOINT, retry=retry)

In [ ]:
# Step 2: Prepare Bucket w/ Tar Data

LIBRISPEECH_URL = "https://www.openslr.org/resources/31/dev-clean-2.tar.gz"
DOWNLOAD_PATH = "./dev-clean-2.tar.gz"
OBJECT_NAME = "librispeech-dev-clean-2.tar.gz"
BUCKET_NAME = "test-librispeech-bucket"

# Step 2a: Download the compressed tar.gz file if it doesn't already exist
if not os.path.exists(DOWNLOAD_PATH):
    response = requests.get(LIBRISPEECH_URL, stream=True, timeout=10)
    with open(DOWNLOAD_PATH, "wb") as f:
        shutil.copyfileobj(response.raw, f)

# Step 2b: Upload the tar.gz file to AIStore
client.bucket(BUCKET_NAME).create(exist_ok=True)
client.bucket(BUCKET_NAME).object(OBJECT_NAME).put_file(DOWNLOAD_PATH)


The `ObjectFile` implementation catches instances of `ChunkedEncodingError` mid-read and retries a new object stream from the last known position to resume safely, where `max_resume` dictates the number of resumes we will allow:

In [ ]:
# Step 3: Read using ObjectFile

with client.bucket(BUCKET_NAME).object(OBJECT_NAME).get().as_file(max_resume=3) as f:
    print(f.read(10))  # Read the first 10 bytes of the file
    print(f.read(10))  # Read the next 10 bytes of the file

`ObjectFile` can be used in any context where a non-seekable, sequential file-like object is expected, such as `tarfile.open` in streaming mode `r|*`:

In [ ]:
EXTRACT_PATH = "./librispeech_extract"
os.makedirs(EXTRACT_PATH, exist_ok=True)

with client.bucket(BUCKET_NAME).object(OBJECT_NAME).get().as_file(max_resume=3) as f:
    with tarfile.open(fileobj=f, mode='r|*') as tar:
        tar.extractall(path=EXTRACT_PATH)

In [ ]:
# Step 5: Clean Up

client.bucket(BUCKET_NAME).delete(missing_ok=True)
os.remove(DOWNLOAD_PATH)
if os.path.exists(EXTRACT_PATH):
    os.system(f"rm -rf {EXTRACT_PATH}")

For more information, please refer to the [Python SDK documentation](https://github.com/NVIDIA/aistore/blob/main/docs/python_sdk.md#object_file).